## Jaccard Similarity
The Jaccard similarity between two sets is defined as the ratio of the volume of their intersection divided by the volume of their union.  $Jaccard(A, B) = \frac{|(A \cap B)|}{|(A \cup B)|}$


In the context of graphs, the neighborhood of a vertex defines its set. The Jaccard similarity weight of each edge represents the strength of connection between vertices based on the relative similarity of their neighbors. For further detail see [Wikipedia](https://en.wikipedia.org/wiki/Jaccard_index)

To compute the Jaccard similarity between all-pairs of vertices connected by an edge in cuGraph use: 
**nvJaccard(G)**
* __G__: A cugraph.Graph object

Returns:
* __jaccard_weights__: cudf.DataFrame with three names columns:
    * jaccard_weights["source"]: The source vertex id.
    * jaccard_weights["destination"]: The destination vertex id.
    * jaccard_weights["jaccard_coeff"]: The jaccard coefficient computed between the source and destination vertex.

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  

In [ ]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [ ]:
# define a function for printing the top most similar vertices
def printMostSimilar(df):
    bestEdge = 0
    
    #find the best
    for i in range(len(df)):
        if df['jaccard_coeff'][i] > df['jaccard_coeff'][bestEdge]:
            bestEdge = i
        
    print("Vertices " + str(df['source'][bestEdge]) + " and " + 
          str(df['destination'][bestEdge]) + " are most similar with score: " 
          + str(df['jaccard_coeff'][bestEdge]))

In [ ]:
# define a function for printing similar vertices based on a threshold
def print_gt_threshold(df, limit):
    
    for i in range(len(df)):
        if df['jaccard_coeff'][i] > limit:
            print("Vertices " + str(df['source'][i]) + " and " + 
                  str(df['destination'][i]) + " are similar with score: " + 
                  str(df['jaccard_coeff'][i]))

### Reading a CSV file using cuDF

In [ ]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/networks/karate-data.csv'

In [ ]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [ ]:
# Let's look at the DataFrame. There should be two columns and 154 records
gdf

In [ ]:
# Look at the first few data records - the output should be two colums src and dst
gdf.head().to_pandas()

### Create a Graph and call the Jaccard analytic

In [ ]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src"], gdf["dst"])

In [ ]:
# How many vertices are in the graph?  Remember that Graph is zero based while teh data start at vertex 1
G.num_vertices()

_The Jaccard test graph has only 34 vertices, so why is the Graph listing 35?_

The Graph is zero-based, meaning that the first vertex ID starts at zero.  The test dataset is 1-based.  Because of that, the Graph object adds an extra isolated vertex with an ID of zero.  Hence the difference in vertex count.  
The next version of cuGraph will add a function to address this issue.  

In [ ]:
# Call cugraph.nvJaccard 
df = cugraph.nvJaccard(G)

In [ ]:
printMostSimilar(df)

Vertex 33 has 12 neighbors, vertex 34 has 16 neighbors.  The share 10 neighbors in common,
jaccard = 10 / (10 + (12 -10) + (16-10)) = 10 / 18 = 0.55

In [ ]:
print_gt_threshold(df, 0.4)

In [ ]:
# Since it is a small graph we can print all scores.
# Notice that only connected vertices are computed
print_gt_threshold(df, 0.0)

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___